In [ ]:
# Energy Infrastructure
# Part 1: Balearic Islands
# Candidate Numbers: 1059922


In [ ]:
# Step 1: Import of libraries 

import pandas as pd
import pandapower as pp
from pandapower import plotting
from pandapower.plotting import simple_plotly, pf_res_plotly
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString
import math
import numba
import seaborn

net = pp.create_empty_network() # creating network

In [ ]:
#Step 2: Creation of Buses

V_l_kv = 132 #kV
V_h_kv = 220 #kV
V_kv=66 #kV

# A total of 18 buses are present in the balearic islands

b0 = pp.create_bus(net,  vn_kv = V_l_kv,  name = "Bus 0", geodata = ((1.430833, 38.91917))) #  Ibiza +Formentura bus (this is the ibaza bus it is a load and a generator power from mallorca )
b1 = pp.create_bus(net,  vn_kv = V_l_kv,  name = "Bus 1_h", geodata = ((2.506454, 39.53525))) # Santa Ponca bus (132)(slack bus that receives power from the mainland)
b2 = pp.create_bus(net,  vn_kv = V_h_kv,  name = "Bus 1_l", geodata = ((2.507937, 39.53662))) # Santa Ponca bus (220)
b3 = pp.create_bus(net,  vn_kv = V_h_kv,  name = "Bus 2", geodata = ((2.549167, 39.58417)))   # Valldurgent bus
b4 = pp.create_bus(net,  vn_kv = V_h_kv,  name = "Bus 3", geodata = ((2.678889, 39.65083)))    # Son Reus 
b5= pp.create_bus(net,  vn_kv = V_h_kv,  name = "Bus 4_h", geodata = ((2.724617, 39.56686)))  # Portol bus (Cas tresorer) (220) 
b5_1 = pp.create_bus(net,  vn_kv = V_kv,  name = "Bus 4_l", geodata = ((2.724617, 39.56686)))  # Portol bus (Cas tresorer) (66)
b6 = pp.create_bus(net,  vn_kv = V_h_kv,  name = "Bus 5", geodata = ((2.744444, 39.60028)))    # Son Orlandis bus 
b7 = pp.create_bus(net,  vn_kv = V_h_kv,  name = "Bus 6", geodata = ((3.039444, 39.67333)))    # Llubi bus
b8 = pp.create_bus(net,  vn_kv = V_h_kv,  name = "Bus 7", geodata = ((3.0925, 39.80944)))     # Murterar bus
b9 = pp.create_bus(net,  vn_kv = V_h_kv,  name = "Bus 8_h", geodata = ((3.158056, 39.58194)))   # Es Bessons bus (220)
b10 = pp.create_bus(net, vn_kv = V_l_kv, name = "Bus 8_l", geodata = ((3.159638, 39.58209)))   # Es Bessons bus (132) 
b11= pp.create_bus(net, vn_kv = V_l_kv, name = "Bus 9", geodata = ((3.427024, 39.73827)))      # Cala Mesquida bus 
b12 = pp.create_bus(net, vn_kv = V_l_kv, name = "Bus 10", geodata = ((3.834227, 39.93175)))     # Cala en Bosch bus
b13 = pp.create_bus(net, vn_kv = V_l_kv, name = "Bus 11", geodata = ((3.855278, 40.00333)))     # Cuitadella bus
b14 = pp.create_bus(net, vn_kv = V_l_kv, name = "Bus 12", geodata = ((4.095556, 39.97694)))     # Es Mercadal bus
b15 = pp.create_bus(net, vn_kv = V_l_kv, name = "Bus 13", geodata = ((4.236667, 39.89111)))     # Dragonera bus 
b16 = pp.create_bus(net, vn_kv = V_l_kv, name = "Bus 14", geodata = ((4.258155, 39.89706)))     # Mao bus




net.bus


In [ ]:
# STEP 3: Creation of loads 

#The load on each bus was calculated proportionally to the population of each island

pp.create_load(net, bus=b0, p_mw = 175.42, q_mvar = 84.96)
pp.create_load(net, bus=b1, p_mw = 113.05, q_mvar = 54.75)
pp.create_load(net, bus=b3, p_mw = 113.05, q_mvar = 54.75)
pp.create_load(net, bus=b4, p_mw = 113.05, q_mvar = 54.75)
pp.create_load(net, bus=b5_1, p_mw = 113.05, q_mvar = 54.75)
pp.create_load(net, bus=b6, p_mw = 113.05, q_mvar = 54.75)
pp.create_load(net, bus=b7, p_mw = 113.05, q_mvar = 54.75)
pp.create_load(net, bus=b8, p_mw = 113.05, q_mvar = 54.75)
pp.create_load(net, bus=b10, p_mw = 113.05, q_mvar = 54.75)
pp.create_load(net, bus=b11, p_mw = 113.05, q_mvar = 54.75)
pp.create_load(net, bus=b12, p_mw = 7.67, q_mvar = 3.71)
pp.create_load(net, bus=b13, p_mw = 32.71, q_mvar = 15.84)
pp.create_load(net, bus=b14, p_mw = 22.24, q_mvar = 10.77)
pp.create_load(net, bus=b15, p_mw = 15.60, q_mvar = 7.55)
pp.create_load(net, bus=b16, p_mw = 31.84, q_mvar = 15.42)



In [ ]:

# Solar Switch 

# By turning ON (1) the PV solar fields are taken into consideration in the power flow analysis
# By turning OFF (0) the PV panels are not used, this represents the current power generation of the balearc islands 

solar = 0 # OFF=0 of ON=1

In [ ]:
# Step 4: Creation of generators

# The total amount of power generated was taken as the peak amount generated on the 9th of August 2019
# The location and power capacity of the storage facilities and the solar farms were iterated until the power flow returned acceptable results
#The total pwoer generated on the 9th of august is equivalent to 1296MW
# Over 24hours this represents 31100 MWh
# The Solar energy is required to meet 80% of that demand, therefor 24880MWh
# Assuming the solar panels generate power at peak capaity for 9hours, the solar panels will need to have a total power capacity of 2760MW
# It is assumed that the storage capacity represents nearly 50% of the power capacity on the island at 1605MW
# Both generation sources on Ibiza, on bus 0, are maintained to supply the rest of the daily energy demand ie 20%
# When the solar panels are added, the grid will have a total power capacity of 2990 MW

if solar==0:
    pp.create_gen(net, bus=b0, p_mw = 218.70, name = "Elvissa, Ibiza")
    pp.create_gen(net, bus=b0, p_mw = 12, name = "San Francesco, Ibiza")
    pp.create_gen(net, bus=b4, p_mw = 303.2, name = "Son Reus, Mallorca")
    pp.create_gen(net, bus=b5, p_mw = 280.05, name = "Cas Tresorer, Mallorca ")
    pp.create_gen(net, bus=b8, p_mw = 369.53, name = "Murterar, Mallorca")
    pp.create_gen(net, bus=b16, p_mw = 113.2, name = "Mahon (Mao), Mernoca")

else:
    pp.create_gen(net, bus=b0, p_mw = 218.70, name = "Elvissa, Ibiza")
    pp.create_gen(net, bus=b0, p_mw = 12, name = "San Francesco, Ibiza")
    pp.create_gen(net, bus=b0, p_mw = 60, name = "Solar Field 1")
    pp.create_gen(net, bus=b4, p_mw = 150, name = "Solar Field 2 ")
    pp.create_gen(net, bus=b5, p_mw = 500, name = "Solar Field 3")
    pp.create_gen(net, bus=b8, p_mw = 520, name = "Solar Field 4 ")
    pp.create_gen(net, bus=b7, p_mw = 570, name = "Solar Field 5 ")
    pp.create_gen(net, bus=b10, p_mw = 550, name = "Solar Field 6 ")
    pp.create_gen(net, bus=b14, p_mw = 400, name ="Solar Field 7")
    pp.create_gen(net, bus=b6, p_mw = 10, name = "Solar Field 8 ")
    pp.create_storage(net, bus=b4, p_mw=350, max_e_mwh= 350*4,name = "Storage 1")
    pp.create_storage(net, bus=b14, p_mw=170, max_e_mwh= 170*4,name = "Storage 2")
    pp.create_storage(net, bus=b3, p_mw=300, max_e_mwh= 300*4,name = "Storage 3")
    pp.create_storage(net, bus=b7, p_mw=340, max_e_mwh= 340*4,name = "Storage 4")
    pp.create_storage(net, bus=b8, p_mw=345, max_e_mwh= 345*4,name = "Storage 5")
    pp.create_storage(net, bus=b5, p_mw=200, max_e_mwh= 200*4,name = "Storage 6")


# the Bus 1 located on Mallorca is a positive generator as it "generates" energy by receiving it from the mainland and sends it into the grid
# it has an angle of 0 and all the other angles are relative to it 
pp.create_ext_grid(net, bus=b1, vm_pu=1.00, name="Slack bus")


In [ ]:
# Step 5: Creation of transformers

# Transformer 1 between bus 1 and bus 2
#Santa Ponca (132kV) / Santa Ponca (220kV)
# Parameters used from 100 MVA 220/110 kV standard type

sn_mva = 320     
vn_hv_kv = 132
vn_lv_kv = 220
vk_percent = 12
vkr_percent = 0.26
pfe_kw = 55
i0_percent = 0.06 
pp.create_transformer_from_parameters(net, hv_bus=b1, lv_bus=b2, sn_mva=sn_mva, vn_hv_kv=vn_hv_kv, vn_lv_kv=vn_lv_kv, vkr_percent=vkr_percent, vk_percent=vk_percent, pfe_kw=pfe_kw, i0_percent=i0_percent)

# Transformer 2 form bus 5 to bus 5.1
# Cas Tresorer (220kV) / Cas Tresorer (66kV)
# Parameters used from 100 MVA 220/110 kV standard type

sn_mva = 480
vn_hv_kv = 220
vn_lv_kv = 66
vk_percent = 12
vkr_percent = 0.26
pfe_kw = 55
i0_percent = 0.06
pp.create_transformer_from_parameters(net, hv_bus=b5, lv_bus=b5_1, sn_mva=sn_mva, vn_hv_kv=vn_hv_kv, vn_lv_kv=vn_lv_kv, vkr_percent=vkr_percent, vk_percent=vk_percent, pfe_kw=pfe_kw, i0_percent=i0_percent)

# Transformer 3 from bus 10 to bus 11
# Bessons (220kV) /Es Bessons (132kV)
# Parameters used from 100 MVA 220/110 kV standard type

sn_mva = 446       
vn_hv_kv = 220
vn_lv_kv = 132
vk_percent = 12
vkr_percent = 0.26
pfe_kw = 55
i0_percent = 0.06
pp.create_transformer_from_parameters(net, hv_bus=b9, lv_bus=b10, sn_mva=sn_mva, vn_hv_kv=vn_hv_kv, vn_lv_kv=vn_lv_kv, vkr_percent=vkr_percent, vk_percent=vk_percent, pfe_kw=pfe_kw, i0_percent=i0_percent)

In [ ]:
net.load

In [ ]:
net.gen

In [ ]:
if solar ==1:
    print(net.storage)

In [ ]:
net.trafo

In [ ]:
# Step 6: Creation of lines


# Creation of the lines geodata

loc_b0=(1.430833, 38.91917)
loc_b1=(2.506454, 39.53525)
loc_b2=(2.507937, 39.53662)
loc_b3=(2.549167, 39.58417)
loc_b4=(2.678889, 39.65083)
loc_b5=(2.724617, 39.56686)
loc_b5_1=(2.724617, 39.56686)
loc_b6=(2.744444, 39.60028)
loc_b7=(3.039444, 39.67333)
loc_b8=(3.0925, 39.80944)
loc_b9=(3.158056, 39.58194)
loc_b10=(3.159638, 39.58209)
loc_b11=(3.427024, 39.73827)
loc_b12=(3.834227, 39.93175)
loc_b13=(3.855278, 40.00333)
loc_b14=(4.095556, 39.97694)
loc_b15=(4.236667, 39.89111)
loc_b16=(4.258155, 39.89706)

r_ohm = 0.07283 # line resistance for Condor cables in ohm per km

# The underwater cables connecting the islands between each other are XLPE submarine cables from ABB
# The inland cables are "Condor" cables

pp.create_line_from_parameters(net,from_bus=b0, to_bus=b1,length_km=126, r_ohm_per_km = 0.053, x_ohm_per_km = 0.119, c_nf_per_km = 1.7e-9, max_i_ka=0.545,geodata = [loc_b0, loc_b1])
pp.create_line_from_parameters(net,from_bus=b2, to_bus=b3,length_km=5.599, r_ohm_per_km = r_ohm, x_ohm_per_km = 0.321, c_nf_per_km = 1e-9, max_i_ka=0.975,geodata = [loc_b2, loc_b3])
pp.create_line_from_parameters(net,from_bus=b3, to_bus=b4,length_km=15.05, r_ohm_per_km = r_ohm, x_ohm_per_km = 0.321, c_nf_per_km = 1e-9, max_i_ka=0.975,geodata = [loc_b3, loc_b4])
pp.create_line_from_parameters(net,from_bus=b4, to_bus=b6,length_km=9.41, r_ohm_per_km = r_ohm, x_ohm_per_km = 0.321, c_nf_per_km = 1e-9, max_i_ka=0.975,geodata = [loc_b4, loc_b6])
pp.create_line_from_parameters(net,from_bus=b3, to_bus=b4,length_km=14.89, r_ohm_per_km = r_ohm, x_ohm_per_km = 0.321, c_nf_per_km = 1e-9, max_i_ka=0.975,geodata = [loc_b3, loc_b4])
pp.create_line_from_parameters(net,from_bus=b6, to_bus=b5,length_km=4.357, r_ohm_per_km = r_ohm, x_ohm_per_km = 0.321, c_nf_per_km = 1e-9, max_i_ka=0.975,geodata = [loc_b6, loc_b5])
pp.create_line_from_parameters(net,from_bus=b6, to_bus=b7,length_km=28.906, r_ohm_per_km = r_ohm, x_ohm_per_km = 0.321, c_nf_per_km = 1e-9, max_i_ka=0.975,geodata = [loc_b6, loc_b7])
pp.create_line_from_parameters(net,from_bus=b4, to_bus=b7,length_km=33.830, r_ohm_per_km = r_ohm, x_ohm_per_km = 0.321, c_nf_per_km = 1e-9, max_i_ka=0.975,geodata = [loc_b4, loc_b7])
pp.create_line_from_parameters(net,from_bus=b7, to_bus=b8,length_km=16.77, r_ohm_per_km = r_ohm, x_ohm_per_km = 0.321, c_nf_per_km = 1e-9, max_i_ka=0.975,geodata = [loc_b7, loc_b8])
pp.create_line_from_parameters(net,from_bus=b7, to_bus=b8,length_km=16.06, r_ohm_per_km = r_ohm, x_ohm_per_km = 0.321, c_nf_per_km = 1e-9, max_i_ka=0.975,geodata = [loc_b7, loc_b8])
pp.create_line_from_parameters(net,from_bus=b7, to_bus=b9,length_km=15.177, r_ohm_per_km = r_ohm, x_ohm_per_km = 0.321, c_nf_per_km = 1e-9, max_i_ka=0.975,geodata = [loc_b7, loc_b9])
pp.create_line_from_parameters(net,from_bus=b7, to_bus=b9,length_km=14.954, r_ohm_per_km = r_ohm, x_ohm_per_km = 0.321, c_nf_per_km = 1e-9, max_i_ka=0.975,geodata = [loc_b7, loc_b9])
pp.create_line_from_parameters(net,from_bus=b10, to_bus=b11,length_km=30.276, r_ohm_per_km = r_ohm, x_ohm_per_km = 0.306, c_nf_per_km = 1e-9, max_i_ka=0.975,geodata = [loc_b10, loc_b11])
pp.create_line_from_parameters(net,from_bus=b11, to_bus=b12,length_km=52.942, r_ohm_per_km = 0.053, x_ohm_per_km = 0.119, c_nf_per_km = 1.7e-9, max_i_ka=0.545,geodata = [loc_b11, loc_b12])
pp.create_line_from_parameters(net,from_bus=b12, to_bus=b13,length_km=8.201, r_ohm_per_km = r_ohm, x_ohm_per_km = 0.306, c_nf_per_km = 1e-9, max_i_ka=0.975,geodata = [loc_b12, loc_b13])
pp.create_line_from_parameters(net,from_bus=b13, to_bus=b15,length_km=35.54, r_ohm_per_km = r_ohm, x_ohm_per_km = 0.306, c_nf_per_km = 1e-9, max_i_ka=0.975,geodata = [loc_b13, loc_b15])
pp.create_line_from_parameters(net,from_bus=b13, to_bus=b14,length_km=21.132, r_ohm_per_km = r_ohm, x_ohm_per_km = 0.306, c_nf_per_km = 1e-9, max_i_ka=0.975,geodata = [loc_b13, loc_b14])
pp.create_line_from_parameters(net,from_bus=b14, to_bus=b15,length_km=15.439, r_ohm_per_km = r_ohm, x_ohm_per_km = 0.306, c_nf_per_km = 1e-9, max_i_ka=0.975,geodata = [loc_b14, loc_b15])
pp.create_line_from_parameters(net,from_bus=b15, to_bus=b16,length_km=4.712, r_ohm_per_km = r_ohm, x_ohm_per_km = 0.306, c_nf_per_km = 1e-9, max_i_ka=0.975,geodata = [loc_b15, loc_b16])





net.line


In [ ]:
# Power Flow Analysis
pp.runpp(net,algorithm='nr');    

In [ ]:
net.res_bus

In [ ]:
net.res_line

In [ ]:
net.res_trafo

In [ ]:
net.res_ext_grid

In [ ]:
net.res_load

In [ ]:
net.res_gen

In [ ]:
simple_plotly(net)

In [ ]:
#Question A

S_base=480000000 #Rated power (Highest Apparent Power between the three transformers)
V_base220=220000 #Base voltage 
V_base132=132000 #Base Voltage

# Calculating the base impedances

Z_base220=V_base220**2/S_base
Z_base132=V_base132**2/S_base

# Creation of the list to place the results
x_pu=list(range(19))
r_pu=list(range(19))
p_pu=list(range(19))

#Arrays of the line parameters

line_info_r_ohm = np.array([0.053,0.07283,0.07283,0.07283,0.07283,0.07283,0.07283,0.07283,0.07283,0.07283,0.07283,0.07283,0.07283,0.053,0.07283,0.07283,0.07283,0.07283,0.07283],dtype=float)
line_info_x_ohm = np.array([0.119,0.321,0.321,0.321,0.321,0.321,0.321,0.321,0.321,0.321,0.321,0.321,0.306,0.119,0.306,0.306,0.306,0.306,0.306],dtype=float)
line_info_voltage = np.array([132,220,220,220,220,220,220,220,220,220,220,220,132,132,132,132,132,132,132],dtype=float)
line_info_maxka= np.array([0.545,0.975,0.975,0.975,0.975,0.975,0.975,0.975,0.975,0.975,0.975,0.975,0.975,0.545,0.975,0.975,0.975,0.975,0.975],dtype=float)

# Calculation of r,x and P in "pu"
for i in range(19):
    if line_info_voltage[i]==132:
        Z_base=Z_base132
    else:
        Z_base=Z_base220
    x_pu[i]=line_info_x_ohm[i]/Z_base*100
    r_pu[i]=line_info_r_ohm[i]/Z_base*100
    p_pu[i]=(line_info_voltage[i]*line_info_maxka[i])/S_base*1E6
    

line_pu=pd.DataFrame({'x_pu':x_pu, 'r_pu':r_pu, 'p_pu':p_pu})
line_pu

In [ ]:
# Question D

from pandapower.plotting.plotly import pf_res_plotly
pf_res_plotly(net)
print(np.sum(net.gen['p_mw']))
print(net.res_bus)

In [ ]:
import seaborn
import matplotlib.pyplot as plt


pp.to_excel(net, 'BI_Part2.xlsx')

colors = seaborn.color_palette()
bc = pp.plotting.create_bus_collection(net, buses=net.res_bus.index, size=.03, color=colors[0], patch_type='circle', cmap='viridis', norm=None, \
                                       infofunc=None, picker=False, bus_geodata=None, cbar_title='Bus Voltage [pu]')
if solar==0:
    lc = pp.plotting.create_line_collection(net, lines=net.res_line.index, line_geodata=None, bus_geodata=None, use_bus_geodata=False,\
                                        infofunc=None, cmap='CMRmap', norm=None, picker=False, z=None, cbar_title='Line Loading [%]', clim=(0,100), plot_colormap=True)
else:
    lc = pp.plotting.create_line_collection(net, lines=net.res_line.index, line_geodata=None, bus_geodata=None, use_bus_geodata=False,\
                                        infofunc=None, cmap='hsv', norm=None, picker=False, z=None, cbar_title='Line Loading [%]', clim=(0,100), plot_colormap=True)


pp.plotting.draw_collections([bc, lc])
pp.plotting.to_html(net, 'ENG_BI_Part2.html', respect_switches=True, include_lines=True, include_trafos=True, show_tables=True)
pp.to_json(net, "ENG_BI_Part2.json")
